# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [4]:
print("Hello World!")

Hello World!


In [1]:
import numpy as np
import torch
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from data import load_and_preprocess_data
from model_setup import get_model
import os
from torchvision.transforms import Compose, Resize, ToTensor
from PIL import Image, ImageDraw, ImageFont

In [2]:
# 训练 PyTorch 模型的函数
def train_torch_model(model, train_loader, val_loader, epochs=10, lr=0.001):
    """
    训练 PyTorch 模型。

    参数：
    - model: 要训练的 PyTorch 模型。
    - train_loader: 用于训练的数据加载器。
    - val_loader: 用于验证的数据加载器。
    - epochs: 训练的总轮数，默认为 10。
    - lr: 学习率，默认为 0.001。

    返回：
    - metrics: 包含验证集的评价指标（准确率、精确率、召回率、F1分数）。
    """
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)  # 使用 Adam 优化器
    criterion = torch.nn.CrossEntropyLoss()  # 使用交叉熵损失函数

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {train_loss / len(train_loader):.4f}")

    # 在验证集上评估模型
    model.eval()
    val_preds, val_targets = [], []
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            val_preds.extend(preds.cpu().numpy())
            val_targets.extend(labels.cpu().numpy())

    # 计算评价指标
    accuracy = accuracy_score(val_targets, val_preds)  # 准确率
    precision = precision_score(val_targets, val_preds, average='weighted')  # 精确率
    recall = recall_score(val_targets, val_preds, average='weighted')  # 召回率
    f1 = f1_score(val_targets, val_preds, average='weighted')  # F1 分数

    metrics = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

    print("Validation Metrics:")
    for key, value in metrics.items():
        print(f"{key.capitalize()}: {value * 100:.2f}%")

    return metrics

In [3]:
def train_non_torch_model(model, x_train, y_train, x_val, y_val):
    """
    训练非 PyTorch 模型（KNN 或 SVM）。

    参数：
    - model: sklearn 模型（KNN 或 SVM）。
    - x_train: 训练集特征。
    - y_train: 训练集标签。
    - x_val: 验证集特征。
    - y_val: 验证集标签。

    返回：
    - metrics: 包含验证集的评价指标（准确率、精确率、召回率、F1分数）。
    """
    model.fit(x_train, y_train)  # 训练模型
    val_preds = model.predict(x_val)  # 在验证集上预测

    # 计算评价指标
    accuracy = accuracy_score(y_val, val_preds)  # 准确率
    precision = precision_score(y_val, val_preds, average='weighted')  # 精确率
    recall = recall_score(y_val, val_preds, average='weighted')  # 召回率
    f1 = f1_score(y_val, val_preds, average='weighted')  # F1 分数

    metrics = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

    print("Validation Metrics:")
    for key, value in metrics.items():
        print(f"{key.capitalize()}: {value * 100:.2f}%")

    return metrics

In [4]:
# 识别并写入图片的函数
def recognize_and_annotate_images(model, folder_path, output_folder, transform, device):
    """
    识别文件夹中的数字或字母，并将识别结果写入图片。

    参数：
    - model: 预训练的 PyTorch 模型。
    - folder_path: 包含图片的文件夹路径。
    - output_folder: 保存注释后图片的文件夹路径。
    - transform: 对图片进行预处理的转换操作。
    - device: 使用的设备（CPU 或 GPU）。
    """
    os.makedirs(output_folder, exist_ok=True)  # 确保输出文件夹存在

    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path).convert('RGB')  # 转换为 RGB 格式
            original_image = image.copy()  # 保留原始图像用于绘制

            # 预处理图片
            processed_image = transform(image).unsqueeze(0).to(device)

            # 使用模型预测
            model.eval()
            with torch.no_grad():
                outputs = model(processed_image)
                predicted = torch.argmax(outputs, dim=1).item()

            # 在图片上写入识别结果
            draw = ImageDraw.Draw(original_image)
            font = ImageFont.load_default()  # 使用默认字体
            text = f"{predicted}"  # 识别结果
            draw.text((10, 10), text, fill=(255, 0, 0), font=font)  # 在左上角写入文字

            # 保存图片到输出文件夹
            output_path = os.path.join(output_folder, filename)
            original_image.save(output_path)
            print(f"Processed and saved: {output_path}")

In [5]:
if __name__ == "__main__":
    # 加载并预处理数据
    x_min_max, x_mean, y, kf_splits = load_and_preprocess_data()

    # 选择一组 K 折数据进行训练和验证
    train_idx, val_idx = kf_splits[0]
    x_train, x_val = x_min_max[train_idx], x_min_max[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # 初始化模型
    model_name = input("请输入要使用的模型 (torch_cnn/knn/svm): ").strip().lower()
    model = get_model(model_name)

    if isinstance(model, torch.nn.Module):  # 判断是否为 PyTorch 模型
        # 转换为 PyTorch 张量
        x_train_tensor = torch.tensor(x_train).float().unsqueeze(1)  # 添加通道维度
        x_val_tensor = torch.tensor(x_val).float().unsqueeze(1)
        y_train_tensor = torch.tensor(y_train).long()
        y_val_tensor = torch.tensor(y_val).long()

        # 创建 DataLoader
        train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
        val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)  # 批大小为 32
        val_loader = DataLoader(val_dataset, batch_size=32)

        # 训练 PyTorch 模型
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        metrics = train_torch_model(model, train_loader, val_loader, epochs=5)  # 训练 5 个轮次

    else:
        # 非 PyTorch 模型（如 KNN 或 SVM）
        metrics = train_non_torch_model(model, x_train.reshape(len(x_train), -1), y_train,
                                        x_val.reshape(len(x_val), -1), y_val)

    # 输出最终验证集评价指标
    print("Final Validation Metrics:")
    for key, value in metrics.items():
        print(f"{key.capitalize()}: {value * 100:.2f}%")

    # 可选：识别 upload 文件夹中的图片，并保存带注释的图片
    recognize_option = input("是否识别 upload 文件夹中的图片并保存结果？(yes/no): ").strip().lower()
    if recognize_option == "yes":
        upload_folder = "./upload"  # upload 文件夹路径
        output_folder = "./output"  # output 文件夹路径
        transform = Compose([
            Resize((64, 64)),  # 将图片调整为 64x64
            ToTensor()        # 转换为 PyTorch 张量
        ])
        recognize_and_annotate_images(model, upload_folder, output_folder, transform, device)

C:\Users\20971\AppData\Local\Temp\ipykernel_448\1957094418.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_train_tensor = torch.tensor(x_train).float().unsqueeze(1)  # 添加通道维度
C:\Users\20971\AppData\Local\Temp\ipykernel_448\1957094418.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_val_tensor = torch.tensor(x_val).float().unsqueeze(1)
C:\Users\20971\AppData\Local\Temp\ipykernel_448\1957094418.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_tensor = torch.tensor(y_train).long()
C:\Users\20971\AppData\Local\Temp\ip

Epoch 1/5, Loss: 0.1431
Epoch 2/5, Loss: 0.0446
Epoch 3/5, Loss: 0.0303
Epoch 4/5, Loss: 0.0216
Epoch 5/5, Loss: 0.0169
Validation Metrics:
Accuracy: 99.01%
Precision: 99.02%
Recall: 99.01%
F1: 99.01%
Final Validation Metrics:
Accuracy: 99.01%
Precision: 99.02%
Recall: 99.01%
F1: 99.01%
